In [1]:
import psycopg2
import pandas as pd
from IPython import display
# get_conn  = lambda: psycopg2.connect(host='10.32.1.107', port=5432, dbname='city_db_final', user='postgres', password='postgres', application_name='select_maintenance')
#conn = psycopg2.connect(host='localhost', port=5432, dbname='city_db_test', user='postgres', password='postgres', application_name='update_maintenance')
conn = psycopg2.connect(host='10.32.1.107', port=5432, dbname='city_db_final', user='postgres', password='postgres', application_name='update_maintenance')
pd.set_option('display.max_rows', 500)
file_export = 'maintenance_export_2023_12_05.xlsx'
file_new = 'ЖС по всем городам.xlsx'
city_service = pd.read_excel(file_export, sheet_name='city_service_types')
social_groups = pd.read_excel(file_export, sheet_name='social_groups')
social_groups= social_groups.rename(columns={'id':'social_group_id','name':'social_group'})
new_lss = pd.read_excel(file_new, sheet_name="living_situations_services")

new_sgvt = pd.read_excel(file_new, sheet_name="social_groups_value_types")



In [2]:
#парсинг ЖС(living_situations) и ЖС-сервисы (living_situations_city_service_types) из нового документа
living_situations = {int(idx): str(name).strip() for _, (idx, name) in new_lss[['id', 'name']].iterrows()}
living_situations = pd.DataFrame(living_situations.items(),columns=['living_situation_id','living_situation'])

display.display(living_situations)
city_service = city_service.rename(columns={'name':'city_service_type','id':'city_service_type_id'})
display.display(city_service)
living_situations_service_types = []
for _, row in new_lss.iterrows():
    for value in row[2:]:
        if value == value:
            living_situations_service_types.append((str(row.iloc[1]).strip(), str(value).strip(), {},''))

living_situations_service_types = pd.DataFrame(living_situations_service_types, columns=['living_situation', 'city_service_type', 'additional','evaluation'])
living_situations_service_types = pd.merge(living_situations_service_types,living_situations,on='living_situation',how='left')

    
living_situations_service_types = pd.merge(living_situations_service_types,city_service,on='city_service_type',how='left')

display.display(living_situations_service_types)

#TODO Сравнить старые с новыми


,living_situation_id,living_situation
0,1,Поездка на общественном транспорте
1,2,Поездка на личном транспорте
2,3,Травма
3,4,Травма ребенка
4,5,Болезнь
5,6,Болезнь ребенка
6,7,Мед. Осмотр
7,8,Мед. Осмотр ребенка
8,9,Лечение зубов
9,10,Психические / психологические проблемы


,city_service_type_id,city_service_type
0,1,Детский сад
1,2,Школа
2,3,Учреждение среднего профессионального образования
3,4,Учреждение высшего профессионального образования
4,5,Поликлиника
5,6,Больница
6,8,Женская консультация
7,9,Роддом
8,10,Стоматология
9,11,Аптека


,living_situation,city_service_type,additional,evaluation,living_situation_id,city_service_type_id
0,Поездка на общественном транспорте,Остановка общественного транспорта,{},,1,113
1,Поездка на общественном транспорте,Станция метро,{},,1,77
2,Поездка на личном транспорте,Автозаправка,{},,2,15
3,Поездка на личном транспорте,Автопарковка,{},,2,61
4,Травма,Травмпункт,{},,3,119
5,Травма,Больница,{},,3,6
6,Травма,Аптека,{},,3,11
7,Травма,Станция скорой помощи,{},,3,69
8,Травма,Лабораторная служба,{},,3,128
9,Травма,Реабилитационный центр,{},,3,137


In [3]:
#Парсинг ценностей(value groups), потребности(value types) и social_groups_value_types_living_situations
sgvt: pd.DataFrame = new_sgvt
sgvt.columns = ['Ценность', 'Потребность'] + list(sgvt.iloc[0][2:])
sgvt = sgvt.iloc[1:]
sgvt = sgvt.rename({
    '(7-11)' : 'Дети младшего школьного возраста (7-11)',
    '(12-15)' : 'Подростки младшего возраста (12-15)',
    '(16-18)' : 'Подростки старшего возраста (16-18)',
    '(19-30)' : 'Молодые горожане (19-30)',
    '(31-40)' : 'Горожане среднего возраста (31-40)',
    '(41-60)' : 'Горожане старшего возраста (41-60)',
    '(61-75)' : 'Люди старшего возраста (61-75)',
    '(старше 75)' : 'Люди преклонного возраста (старше 75)',
    'БЖ' : 'Беременные женщины',
    'ГМ' : 'Горожане с младенцами',
    'ГД (1-3 года)' : 'Горожане с детьми детсадовского возраста (1-3 года)',
    'ГД (3-6 лет)' : 'Горожане с детьми дошкольного возраста (3-6 лет)',
    'ГД (7-11 лет)' : 'Горожане с детьми младшего школьного возраста (7-11 лет)',
    'ГД (12-18 лет)' : 'Горожане с подростками (12-18 лет)',
    'СУ' : 'Студенты и учащиеся колледжей'
}, axis=1)
value_groups = sgvt["Ценность"].dropna().reset_index(drop=True)
value_groups = pd.DataFrame(value_groups)
value_groups = value_groups.rename(columns={'Ценность':'group_name'})
value_groups['group_id'] = range(1,len(value_groups)+1)

sgvt['Ценность'] = sgvt['Ценность'].ffill()

value_types = {group: type_ for _,(type_,group) in sgvt[['Ценность','Потребность']].iterrows() }

value_types = pd.DataFrame(value_types.items(),columns=['value_type','group_name'])
value_types = pd.merge(value_types,value_groups,how='left',on='group_name')
value_types['value_type_id'] = range(1,len(value_types)+1)

social_groups_value_types_living_situations = []
for _, row in sgvt.iterrows():
    for social_group, values in zip(sgvt.columns[2:], row[2:]):
        value_type = row.iloc[1]
        values = str(values) if values == values else ''
        if len(values.strip()) == 0:
            continue
        values = list(map(str.strip, values.split(',')))
        for value in values:
            social_groups_value_types_living_situations.append((value_type, social_group, value))
social_groups_value_types_living_situations = pd.DataFrame(social_groups_value_types_living_situations, columns=['value_type', 'social_group', 'living_situation'])

social_groups_value_types_living_situations = pd.merge(social_groups_value_types_living_situations,value_types[['value_type','value_type_id']],how='left',on='value_type')
social_groups_value_types_living_situations = pd.merge(social_groups_value_types_living_situations,social_groups[['social_group','social_group_id']],how='left',on='social_group')
social_groups_value_types_living_situations = pd.merge(social_groups_value_types_living_situations,living_situations,how='left',on='living_situation')
display.display(value_groups)
display.display(value_types)
display.display(social_groups_value_types_living_situations)

,group_name,group_id
0,Защищенности,1
1,Общего благополучия,2
2,Развития,3


,value_type,group_name,group_id,value_type_id
0,Обеспечение физического здоровья граждан,Защищенности,1,1
1,Обеспечение социальной защищенности,Защищенности,1,2
2,Обеспечение возможности социальной и профессио...,Общего благополучия,2,3
3,Обеспечение ежедневных потребностей,Общего благополучия,2,4
4,"Обеспеченние культурной, творческой и досугово...",Развития,3,5
5,Возможность улучшения условий жизни и развития,Развития,3,6


,value_type,social_group,living_situation,value_type_id,social_group_id,living_situation_id
0,Обеспечение физического здоровья граждан,Дети младшего школьного возраста (7-11),Травма,1,21,3
1,Обеспечение физического здоровья граждан,Дети младшего школьного возраста (7-11),Болезнь,1,21,5
2,Обеспечение физического здоровья граждан,Дети младшего школьного возраста (7-11),Мед. Осмотр,1,21,7
3,Обеспечение физического здоровья граждан,Дети младшего школьного возраста (7-11),Лечение зубов,1,21,9
4,Обеспечение физического здоровья граждан,Подростки младшего возраста (12-15),Травма,1,22,3
5,Обеспечение физического здоровья граждан,Подростки младшего возраста (12-15),Болезнь,1,22,5
6,Обеспечение физического здоровья граждан,Подростки младшего возраста (12-15),Мед. Осмотр,1,22,7
7,Обеспечение физического здоровья граждан,Подростки младшего возраста (12-15),Лечение зубов,1,22,9
8,Обеспечение физического здоровья граждан,Подростки старшего возраста (16-18),Травма,1,26,3
9,Обеспечение физического здоровья граждан,Подростки старшего возраста (16-18),Болезнь,1,26,5


# Truncate

In [4]:
with conn, conn.cursor() as cur:
    cur.execute(
        "truncate maintenance.value_groups cascade"
    )
    cur.execute(
        "truncate living_situations cascade"
    )

# Update

In [5]:
with  conn, conn.cursor() as cur:
    for _,row in living_situations.iterrows():
        cur.execute(
            "INSERT  INTO living_situations (id, name) VALUES (%s, %s)",(row['living_situation_id'],row['living_situation'])
        )


In [6]:
with  conn, conn.cursor() as cur:
    for _,row in living_situations_service_types.iterrows():
        cur.execute(
            "INSERT INTO maintenance.living_situations_city_service_types (living_situation_id, city_service_type_id,additional) VALUES (%s, %s, DEFAULT)",(row['living_situation_id'],row['city_service_type_id'])
        )

In [7]:
with  conn, conn.cursor() as cur:
    for _,row in value_groups.iterrows():
        cur.execute(
            "INSERT INTO maintenance.value_groups (id, name) VALUES (%s, %s)",(row['group_id'],row['group_name'])
        )
    for _,row in value_types.iterrows():
        cur.execute(
            "INSERT INTO maintenance.value_types (id, name,group_id) VALUES (%s, %s, %s)",(row['value_type_id'],row['value_type'],row['group_id'])
        )

In [8]:
with  conn, conn.cursor() as cur:
    for _,row in social_groups_value_types_living_situations.iterrows():
        cur.execute(
            "INSERT INTO maintenance.social_groups_value_types_living_situations (social_group_id, value_type_id,living_situation_id) VALUES (%s, %s, %s)",(row['social_group_id'],row['value_type_id'],row['living_situation_id'])
        )